# Support

If you have any questions or want to report an issue regarding the Python SDK please email <a href="mailto:clarion-support@coremont.com?subject=Clarion API/SDK Support Query&body=Your Email Account for the API (if not your own):%0D%0ADate/time of the issue:%0D%0ARelevant section of the log file:%0D%0ASDK Version:%0D%0AWhat you expected to happen and what actually happened:%0D%0AAny other relevant information:%0D%0A">Clarion Support</a> with the following information:

- Your email account for the API (in the case that it is not the same as the originating email address),
- Date/time of the issue,
- The relevant section of the log file (see [Logging](https://cm.coremont.com/docs/api/sdk/details#logging) section) at the time the issue occurred, which will contain the [version](https://cm.coremont.com/docs/api/sdk/details#versioning) and correlation ID,
- What you expected to happen and what actually happened,
- Any other relevant information.

## Typical trade pricing worklow

1.  Create a trade by supplying the mandatory parameters required by the
    specific trade type using the
    **[create_trade](https://cm.coremont.com/docs/api/sdk/functions#create_trade)**
    function. The list of the mandatory parameters for each trade type
    can be found
    [here](https://cm.coremont.com/docs/api/sdk/functions#create_trade).
    The applicable values for each instrument can be obtained from the
    Clarion UI line **Pricer** screen.
2.  Identify the set of price metrics of interest. The
    [metrics](https://cm.coremont.com/docs/api/sdk/functions#metrics)
    function can be used to get a list of all supported metrics.
3.  Get the list of price metrics for the trade for current or series of
    historical dates using the
    **[price](https://cm.coremont.com/docs/api/sdk/functions#price)**
    function.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv();

In [ ]:
import clarion
from gioutils.clarion_ext import super_clarion

clarion.login()

In [ ]:
super_clarion(clarion)

In [ ]:
# See what metrics and options for config are available
clarion.price_metrics();

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
m1 = clarion.metric('PresentValue', 'PresentValue', 'Any')

m2 = clarion.metric('Price', 'Price', 'Fx', {'Unit': 'PercentLocal'}) 
m3 = clarion.metric('AtmVol','Volatility','Fx',{'Type': 'ATM'})
m4 = clarion.metric('StrikeVol','Volatility','Fx',{'Type': 'Strike'})
m5 = clarion.metric('T+1Carry','Carry','Fx',{'Period': '1', 'StartBumpDays': '1'})

m6 = clarion.metric('FwdRate', 'Forward', 'Rates')
m7 = clarion.metric('StrikeVol', 'Volatility', 'Rates',{'Type': 'Strike', 'Period': 'Daily'})

m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [m1,m2,m3,m4,m5]
rates_metrics = [m1,m6,m7]
eq_metrics = [m1,m8,m9]

## Find which products are supported and required parameters

In [ ]:
instrument_types = clarion.price.instrument_types()
instrument_types;

In [ ]:
inst_type_params = clarion.price.instrument_type_params('FxVanilla')
inst_type_params;

In [ ]:
fx_vanilla_1 = clarion.instruments.FxVanilla(
    currency_pair='USDMXN',
    expiry='3m',
    strike='f',              
    option_type='Call'
)

In [ ]:
#create trade
fx_vanilla_1 
fx_trades = [fx_vanilla_1.trade] 

In [ ]:
response = clarion.price.run(fx_metrics, fx_trades, market_mode='live')

In [ ]:
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
# historic pricing returning specified fx metrics, note historic pricing based on London 4pm close
response = clarion.price.run(fx_metrics, fx_trades, start_date="2021-06-30",end_date="2022-06-30", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Fx Reverse Knockout pricing

In [ ]:
fx_rko_1 = clarion.instruments.FxReverseKnockOut(
    currency_pair='USDMXN',
    expiry='3m',
    strike='21',              
    option_type='Call',
    barrier_up='26'
)
response = clarion.price.run(fx_metrics, fx_rko_1.trade, start_date="2022-03-01",end_date="2022-03-16", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Rates Swap pricing

In [ ]:
swap_1 = clarion.instruments.RatesSwap(
    currency='USD',
    instrument_name='LIBOR S 3M',
    strike='a',
    tenor='10y',
    start_date='1y',
    swap_type='Pay',
    # notional='d20000',
    clearing_house='LCH',
)
response = clarion.price.run(rates_metrics, swap_1.trade, market_mode='live')

In [ ]:
# response = clarion.price.run(rates_metrics, swap_1.trade, start_date="2022-03-01",end_date="2022-03-16", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
# response = clarion.price.run(rates_metrics, swap_1.trade, start_date="2022-03-01",end_date="2022-03-16", market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Rates swaption pricing

In [ ]:
rates_swaption = clarion.create_trade('swaption_1', 'ratesSwaption', {'InstrumentName': 'BBSW Q 3M',
                                                                 'Currency': 'AUD',
                                                                 'Expiry': '10-Mar-23',
                                                                 'Notional': 100000000,
                                                                 'Tenor':'6M',
                                                                 'Strike': '0.2',
                                                                 'SwaptionType': 'Pay',
                                                                 'CollateralCurrency': 'USD',
                                                                 'SettleType': 'Physical Cleared LCH',
                                                                 'PaySettle': 'Spot'})

response = clarion.price.run(rates_metrics, rates_swaption, start_date="2022-03-01",end_date="2022-03-16",market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Equity future option pricing

In [ ]:
equity_futOpt = clarion.create_trade('id2','equityFutureOption',{'InstrumentName':'AS51 SFE',
                                                         'UnderlyingExpiry':'Sep23',
                                                         'ContractType':'OTC', 
                                                         'Expiry':'2023-09-21',
                                                         'Strike':'5500',
                                                         'OptionType':'C',
                                                         'ExerciseType':'A',
                                                         'Quantity':100})
response = clarion.price.run(eq_metrics, equity_futOpt, start_date="2022-06-01",end_date="2022-09-01",market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

## Retrieve Exchange close pricing
#### Exchange close pricing is available back up to **4w history** from todays date

In [ ]:
rates_STIRFut = clarion.create_trade('stir1', 'ratesShortTermFuture', {'InstrumentName':'EURIBOR 3M ICE',
                                                                       'Currency':'EUR',
                                                                       'Tenor':'H23'})

In [ ]:
m1 = clarion.metric('FwdRate', 'Forward', 'Rates')
response = clarion.price.run([m1], rates_STIRFut, start_date='2022-09-01', end_date='2022-09-05',market_mode='reval',settings={'market_name':'exchange'})
response['DFRAME']